In [1]:
from scrape.wp_get_page import WikipediaPage
from scrape.wp_structured_text import WikipediaStructuredText
from scrape.wp_structured_diffs import Diff
import difflib
from setup.db_trie import db
from lakat.submit import content_submit
from config.scrape_cfg import WIKIPEDIA_API_URL, EXAMPLE_ARTICLE_TITLE
from config.bucket_cfg import DEFAULT_ATOMIC_BUCKET_SCHEMA, DEFAULT_MOLECULAR_BUCKET_SCHEMA, DEFAULT_NAME_RESOLUTION_BUCKET_SCHEMA, BUCKET_ID_TYPE_NO_REF
from utils.signing.sign import get_public_key_from_file
from utils.serialize import unserialize, serialize
from scrape.wp_structured_diffs import _similar_content

# db.close()
article_title = EXAMPLE_ARTICLE_TITLE
wp = WikipediaPage(WIKIPEDIA_API_URL)
edit_history = wp.load_content_from_batches(
    article_title, 0, 105, download_if_not_exist=True)
edit_history.reverse()


In [ ]:
# inds = list()
# for ind in range(len(edit_history) - 1): 
ind = 53 ##65
str_text_old = WikipediaStructuredText(edit_history[ind]["*"])
str_text_new = WikipediaStructuredText(edit_history[ind+1]["*"])
diff = Diff(old=str_text_old, new=str_text_new)
df = diff.compare(similarity_threshold=0.75, zero_level_similarity_threshold=0.95)
# if len(df["modified"]) != 0 and len(df["rearranged"])!=0:
#     inds.append(ind)
df